In [ ]:
import re
import os

FOLDER = "cases"

import json
import pandas as pd
from dataclasses import dataclass

def subtract_df(df_original, df_subset):
    # Слияние двух DataFrame с помощью внешнего слияния (outer join) и индикатора
    merged_df = pd.merge(df_original, df_subset, how='outer', indicator=True)

    # Выбираем только те строки, которые присутствуют в оригинальном DataFrame
    df_result = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

    # Если вы хотите сохранить только столбцы оригинального DataFrame
    df_result = df_result[df_original.columns]

    return df_result



def parse_conditions_to_code(conditions):
    python_code = conditions.replace("AND", "&").replace("OR", "|").replace("NOT", "~")

    python_code = python_code.replace('"Площадка"', 'df_orig["Площадка"]')
    python_code = python_code.replace('"Тип используемых прав" ', 'df_orig["Тип используемых прав"]')
    python_code = python_code.replace('"Территория"', 'df_orig["Территория"]')
    python_code = python_code.replace('"Тип контента"', 'df_orig["Тип контента"]')
    python_code = python_code.replace('"Вид использования  контента"', 'df_orig["Вид использования  контента"]')
    python_code = python_code.replace('"Исполнитель"', 'df_orig["Исполнитель"]')
    python_code = python_code.replace('"Название трека"', 'df_orig["Название трека"]')
    python_code = python_code.replace('"Название альбома"', 'df_orig["Название альбома"]')
    python_code = python_code.replace('"Автор слов"', 'df_orig["Автор слов"]')
    python_code = python_code.replace('"Автор музыки"', 'df_orig["Автор музыки"]')
    python_code = python_code.replace('"ISRC контента"', 'df_orig["ISRC контента"]')
    python_code = python_code.replace('"UPC альбома"', 'df_orig["UPC альбома"]')
    python_code = python_code.replace('"Копирайт"', 'df_orig["Копирайт"]')

    python_code = re.sub(r'\s+', ' ', python_code).strip()

    python_code = f'df_orig[{python_code}]'

    return python_code


def calc_sum(df):
    res = 0
    for index, row in df.iterrows():
        res += (row.iloc[c["Вознаграждение_1"]] + row.iloc[c["Вознаграждение_2"]])
    return round(res, 2)


@dataclass
class Data:
    license: int
    track: int
    name: str

def parse_filename(name: str) -> Data:
    name = name.replace(".txt", "")
    d = name.split(",")

    license = int(d[0].replace("license=", ""))
    track = int(d[1].replace("track=", ""))
    name = d[2].replace("name=", "")

    return Data(license=license, track=track, name=name)

with open("config.json", "r", encoding="utf-8") as f:
    c = json.loads(f.read())

df_orig = pd.read_excel(c["filename"], header=c["header_index_from_zero"])

col_count = len(df_orig.columns)
if  col_count > c["col_count"]:
    df_orig = df_orig.iloc[:, :c["col_count"]-col_count]


all_case_dfs = list()
case_dfs = dict()

for case in os.listdir(FOLDER):
    if ".DS_Store" in case:
        continue
    case_path = os.path.join(FOLDER, case)
    res = 0
    
    case_dfs[case] = list()
    
    if os.path.isfile(case_path):
        with open(case_path, "r", encoding="utf-8") as f:
            conditions = f.read()
        code_output = parse_conditions_to_code(conditions)
        data = parse_filename(case)
        df = eval(code_output)

        all_case_dfs.append(df)
        case_dfs[case].append(df)

        res += calc_sum(df) * (data.per/100)
        print(round(res, 2), data.name)
    else:
        for subcase in os.listdir(case_path):
            r = 0
            with open(os.path.join(case_path, subcase), "r", encoding="utf-8") as f:
                conditions = f.read()
            code_output = parse_conditions_to_code(conditions)
            df = eval(code_output)

            all_case_dfs.append(df)
            case_dfs[case].append(df)

            data = parse_filename(subcase)
            r = calc_sum(df) * (data.per/100)

            res += r
        data.name = case

        print(round(res, 2), data.name)



In [2]:
import json 
import pandas as pd 

with open("config.json", "r", encoding="utf-8") as f:
    c = json.loads(f.read())

df_orig = pd.read_excel(c["filename"], header=c["header_index_from_zero"])

In [5]:
print(df_orig[df_orig["ISRC контента"] == "RUA5S2300565"])

Empty DataFrame
Columns: [N, Период использования контента, Площадка, Тип используемых прав, Территория, Тип контента, Вид использования  контента, Исполнитель, Название трека, Название альбома, Автор слов, Автор музыки, Доля авторских прав Лицензиара, Доля смежных прав Лицензиара, ISRC контента, UPC альбома, Копирайт, Количество загрузок/прослушиваний, Сумма денежных средств, полученных ЛИЦЕНЗИАТОМ за авторские права в руб., без НДС, Сумма денежных средств, полученных ЛИЦЕНЗИАТОМ за смежные права в руб., без НДС, Доля Лицензиара, Вознаграждение ЛИЦЕНЗИАРА за авторские права в руб., без НДС, Вознаграждение ЛИЦЕНЗИАРА за смежные права в руб., без НДС, Итого вознаграждение ЛИЦЕНЗИАРА в руб., без НДС, Код лицензиара, Unnamed: 25, Unnamed: 26, код Лицензиата, Договор ID]
Index: []

[0 rows x 29 columns]


In [2]:
import re

def parse_conditions_to_code(conditions):
    python_code = conditions.replace("AND", "&").replace("OR", "|").replace("NOT", "~")

    python_code = python_code.replace('"Площадка"', 'df_orig["Площадка"]')
    python_code = python_code.replace('"Тип используемых прав" ', 'df_orig["Тип используемых прав"]')
    python_code = python_code.replace('"Территория"', 'df_orig["Территория"]')
    python_code = python_code.replace('"Тип контента"', 'df_orig["Тип контента"]')
    python_code = python_code.replace('"Вид использования  контента"', 'df_orig["Вид использования  контента"]')
    python_code = python_code.replace('"Исполнитель"', 'df_orig["Исполнитель"]')
    python_code = python_code.replace('"Название трека"', 'df_orig["Название трека"]')
    python_code = python_code.replace('"Название альбома"', 'df_orig["Название альбома"]')
    python_code = python_code.replace('"Автор слов"', 'df_orig["Автор слов"]')
    python_code = python_code.replace('"Автор музыки"', 'df_orig["Автор музыки"]')
    python_code = python_code.replace('"ISRC контента"', 'df_orig["ISRC контента"]')
    python_code = python_code.replace('"UPC альбома"', 'df_orig["UPC альбома"]')
    python_code = python_code.replace('"Копирайт"', 'df_orig["Копирайт"]')

    python_code = re.sub(r'\s+', ' ', python_code).strip()

    python_code = f'df_orig[{python_code}]'

    return python_code

with open("cases\\name=Phlora,license=70,track=100.txt", "r", encoding="utf-8") as f:
    conditions = f.read()

print([conditions])

code_output = parse_conditions_to_code(conditions)

print(code_output)


['(\n"Название альбома" == "Красивый садист"\n)\nOR\n(\n(\n"Название альбома" == "Волны"\n)\nAND\n(\n"UPC альбома" == "5057827969268"\n)\n)\nOR\n(\n(\n"Название альбома" == "Правда"\n)\nAND\n(\n"UPC альбома" == "5057827969183"\n)\n)\nOR\n(\n"Название альбома" == "Phlora"\n)\nOR\n(\n(\n"Название альбома" == "Breeze"\n)\nAND\n(\n"UPC альбома" == "5057827821948"\n)\n)\nOR\n(\n(\n"Название альбома" == "Навеки"\n)\nAND\n(\n"UPC альбома" == "5057827822136"\n)\n)']
df_orig[( df_orig["Название альбома"] == "Красивый садист" ) | ( ( df_orig["Название альбома"] == "Волны" ) & ( df_orig["UPC альбома"] == "5057827969268" ) ) | ( ( df_orig["Название альбома"] == "Правда" ) & ( df_orig["UPC альбома"] == "5057827969183" ) ) | ( df_orig["Название альбома"] == "Phlora" ) | ( ( df_orig["Название альбома"] == "Breeze" ) & ( df_orig["UPC альбома"] == "5057827821948" ) ) | ( ( df_orig["Название альбома"] == "Навеки" ) & ( df_orig["UPC альбома"] == "5057827822136" ) )]


In [ ]:
import os

def rename_files(directory):
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.txt'):
                # Разделяем имя файла на части
                parts = filename.split(',')
                if len(parts) == 3:
                    # Извлекаем и меняем местами 'track' и 'license'
                    name_part, track_part, license_part = parts
                    track_part, license_part = license_part, track_part

                    # Собираем новое имя файла
                    new_filename = f"{name_part},{track_part[:-4]},{license_part}.txt"

                    # Полный путь к старому и новому файлу
                    old_filepath = os.path.join(root, filename)
                    new_filepath = os.path.join(root, new_filename)

                    # Переименовываем файл
                    os.rename(old_filepath, new_filepath)
                    print(f"Renamed '{old_filepath}' to '{new_filepath}'")

# Путь к директории 'cases'
cases_directory = 'cases'

# Вызываем функцию для переименования файлов
rename_files(cases_directory)


In [ ]:
with open("config.json", "r", encoding="utf-8") as f:
    c = json.loads(f.read())

df_orig = pd.read_excel(c["filename"], header=c["header_index_from_zero"])

col_count = len(df_orig.columns)
if  col_count > c["col_count"]:
    df_orig = df_orig.iloc[:, :c["col_count"]-col_count]

In [ ]:
print(df_orig[(df_orig["Исполнитель"].isna()) & (df_orig["Название трека"] == "Грустные песни")])

In [ ]:
import os

def rename_files_recursively(start_dir):
    for dirpath, _, filenames in os.walk(start_dir):
        for filename in filenames:
            if filename.endswith('.txt') and ',' in filename:
                # Extract the base name without the extension
                base_name = filename[:-4]
                parts = base_name.split(',')
                # Find the part that starts with "name="
                name_part = next((part for part in parts if part.startswith('name=')), None)
                if name_part:
                    # Remove the name part from the list
                    parts.remove(name_part)
                    # Construct the new filename with the name part at the beginning
                    new_base_name = f"{name_part},{','.join(parts)}"
                    new_filename = f"{new_base_name}.txt"
                    old_file_path = os.path.join(dirpath, filename)
                    new_file_path = os.path.join(dirpath, new_filename)
                    # Rename the file
                    os.rename(old_file_path, new_file_path)
                    print(f"Renamed '{old_file_path}' to '{new_file_path}'")

# Replace 'your_directory' with the path to the directory containing your files
rename_files_recursively('cases')



In [ ]:
import pandas as pd

df_orig = pd.read_excel("report.xlsx", header=5)


In [ ]:
import re

def parse_conditions_to_code(conditions):
    python_code = conditions.replace("AND", "&").replace("OR", "|").replace("NOT", "~")

    python_code = python_code.replace('"Площадка"', 'df_orig["Площадка"]')
    python_code = python_code.replace('"Тип используемых прав" ', 'df_orig["Тип используемых прав"]')
    python_code = python_code.replace('"Территория"', 'df_orig["Территория"]')
    python_code = python_code.replace('"Тип контента"', 'df_orig["Тип контента"]')
    python_code = python_code.replace('"Вид использования  контента"', 'df_orig["Вид использования  контента"]')
    python_code = python_code.replace('"Исполнитель"', 'df_orig["Исполнитель"]')
    python_code = python_code.replace('"Название трека"', 'df_orig["Название трека"]')
    python_code = python_code.replace('"Название альбома"', 'df_orig["Название альбома"]')
    python_code = python_code.replace('"Автор слов"', 'df_orig["Автор слов"]')
    python_code = python_code.replace('"Автор музыки"', 'df_orig["Автор музыки"]')
    python_code = python_code.replace('"ISRC контента"', 'df_orig["ISRC контента"]')
    python_code = python_code.replace('"UPC альбома"', 'df_orig["UPC альбома"]')
    python_code = python_code.replace('"Копирайт"', 'df_orig["Копирайт"]')

    python_code = re.sub(r'\s+', ' ', python_code).strip()

    python_code = f'df_orig[{python_code}]'

    return python_code

case_path = "cases/Вика Радисева/name=6,license=100,track=50.txt"

with open(case_path, "r", encoding="utf-8") as f:
    conditions = f.read()

text = parse_conditions_to_code(conditions)

print(text)

df_orig2 = df_orig[( pd.isna(df_orig["Исполнитель"]) ) & ( df_orig["Тип контента"] == "Трек" ) & ( df_orig["Название трека"] == "Грустные песни" )]

print(df_orig2)